In [50]:
from azure.storage.blob import BlobServiceClient
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime

In [ ]:
load_dotenv('../.env')

True

In [52]:
TASK_NAME = "source_parsing_v0"

INPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/"
run_n = len(os.listdir(INPUT_DATA_PATH))
RUNID = f"RUNID_{run_n}"
INPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/BATCHFILES"

RUN_TIME = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')

OUTPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/BATCHID"

os.makedirs(OUTPUT_DATA_PATH, exist_ok=True)

print(f"Run ID: {RUNID} at {RUN_TIME}")

Run ID: RUNID_1 at 2025-05-29 16:18:07


In [53]:
AZURE_OPENAI_API_KEY=os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT=os.getenv('AZURE_OPENAI_ENDPOINT')
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [54]:
def get_batchfiles():
    batchfiles = []
    for filename in os.listdir(INPUT_DATA_PATH):
        if filename.endswith('.jsonl'):
            b = {
                'path': os.path.join(INPUT_DATA_PATH, filename),
                'id' : filename.split('.')[0].split('_')[-1],
            }
            batchfiles.append(b)
    return sorted(batchfiles)

batchfiles = get_batchfiles()

In [55]:
def save_batchid(i, batch_id):
    batchid_filename = f"{RUNID}--{TASK_NAME}_BATCHID_{i}.txt"
    with open(os.path.join(OUTPUT_DATA_PATH, batchid_filename), "w") as f:
        f.write(batch_id)

In [56]:

for bathcfile in batchfiles:

  i = bathcfile['id']
  path = bathcfile['path']

  print("Submitting {}".format(path))

  file = client.files.create(
    file=open(path, "rb"),
    purpose="batch"
  )

  print(file.model_dump_json(indent=2))
  file_id = file.id

  # Submit a batch job with the file
  batch_response = client.batches.create(
      input_file_id=file_id,
      endpoint="/chat/completions",
      completion_window="24h",
  )

  # Save batch ID for later use
  batch_id = batch_response.id

  print(batch_response.model_dump_json(indent=2)) 

  save_batchid(i, batch_id)
  print(f"Batch ID for file {i}: {batch_id}")
  print("Batch job submitted successfully.")
  print("  ")

Submitting ../local_tests_data/azure_openai_batch_processing_files/RUNID_1/source_parsing_v0/BATCHFILES/RUNID_1--source_parsing_v0_BATCHFILE_0.jsonl
{
  "id": "file-d6ac18fed60144819337e6862af93b5c",
  "bytes": 94717,
  "created_at": 1748528289,
  "filename": "RUNID_1--source_parsing_v0_BATCHFILE_0.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
{
  "id": "batch_26b72390-58df-4c33-a3e5-c0dfa4451538",
  "completion_window": "24h",
  "created_at": 1748528295,
  "endpoint": "/chat/completions",
  "input_file_id": "file-d6ac18fed60144819337e6862af93b5c",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1748614691,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0